- https://sage-kite-7e1.notion.site/reinforce-plus-plus
- https://www.bilibili.com/video/BV1eg4y1s7TN
    - [pytorch 强化学习] 13 基于 pytorch 神经网络实现 policy gradient（REINFORCE）求解 CartPole
- https://medium.com/@2468086464/understanding-reinforcement-learning-from-human-feedback-rlhf-theory-and-the-mechanism-ef45485a5070

## PG

$$
\mathbb E_{\tau \sim p_\theta(\tau)}[R(\tau)]
$$

- 求其 gradient

$$
\begin{split}
\sum_{\tau}R(\tau)\nabla p_\theta(\tau)&=\sum_\tau R(\tau) p_\theta(\tau)\nabla \log p_\theta(\tau)\\
&=\mathbb E_{\tau\sim p_\theta(\tau)}R(\tau)\nabla \log p_\theta(\tau)
\end{split}
$$

- 对 $\nabla \log p_\theta(\tau)$ 进行展开

$$
\begin{split}
\nabla \log p_\theta(\tau)&=\nabla \log\left(p(s_1)\prod p(a_t|s_t)\prod p(s_{t+1}|s_t,a_t)\right)\\
&=\nabla \left(\log p(s_1)+\sum \log p(a_t|s_t)+\sum \log p(s_{t+1}|s_t,a_t)\right)\\
&=\sum_{t=1}^T\nabla \log p(a_t|s_t)
\end{split}
$$

- 因此
$$
\begin{split}
\sum_{\tau}R(\tau)\nabla p_\theta(\tau)&=\sum_\tau R(\tau) p_\theta(\tau)\nabla \log p_\theta(\tau)\\
&=\mathbb E_{\tau\sim p_\theta(\tau)}R(\tau)\nabla \log p_\theta(\tau)\\
&\approx\frac1N\sum_{n=1}^NR(\tau^n)\nabla \log p_\theta(\tau^n)\\
&=\frac1N\sum_{n=1}^N\sum_{t=1}^{T_n}R(\tau^n)\nabla \log p_\theta(a_t^n|s_t^n)
\end{split}
$$


### PG vs. REINFORCE

> REINFORCE is a crutial and simple PG method in RL designed to maximize expected cumulative rewards through direct PO (policy optimization).
> > **RE**ward **I**ncrement = **N**onnegative **F**actor × **O**ffset **R**einforcement × **C**haracteristic **E**ligibility.”

- discounted cumulative future reward
    - $G_t=\sum_{k=t+1}^{T}\gamma^{k-t}r_k$
    - $r_t$: the immediate reward at time step $t$

$$\nabla_\theta J(\theta)=\mathbb E_{\tau\sim\pi_\theta}[G_t\nabla_\theta \log\pi_\theta(A_t|S_t)]$$

- $G(\tau)\rightarrow G_t$
- PG: $G(\tau)=r_0+\gamma r_1+\gamma^2 r_2+\cdots=\sum_{t=0}^T\gamma^tr_t$
    - $G(\tau)\nabla_\theta\log\pi_\theta(A_t|S_t)$，无论在哪个时刻 $t$，我们始终都会使用固定不变的权重 $G(\tau)$ 来增加或减少采取行动 $A_t$ 的条件概率；
- REINFORCE: $G_t=\sum_{k=t+1}^{T}\gamma^{k-t}r_k$
    - agent 行动的好坏是根据行动之后获得的奖励综合来评估的，采取某个行动之前获得的奖励与该行动的好坏无关；

### baseline

- minus baseline
    - 在一些坏的局势下，做什么动作都会得到负的奖励。所以要减去baseline。（即坏的局势下，仍然有好的 action，比如走出当前困境的 action）

## REINFORCE++

> 为了稳定训练

- token-level KL-penalty
    - $r(s_t,a_t)=\mathbb I(s_t=[EOS])r(x,y)-\beta KL(t)$
    - The advantage of this Token-level KL is that it seamlessly integrates with Process Reward Models (PRM) and achieves credit assignment, which is only necessary to add $r^{\text{process}}(s_t, a_t)$at the position of the reward token.
        - Recently, some resachers discovered that using an external KL loss (不直接定义在 token 级别的 reward 上) with GRPO for REINFORCE++ also works.
- Mini-batch Updates
- Reward Normalization and Clipping
- Advantage Normalization
    - $A_t(s_t, a_t) = r(x, y) - \beta \cdot \sum_{i=t}^{T} \text{KL}(i)$
 
https://github.com/OpenRLHF/OpenRLHF/blob/main/examples/scripts/train_reinforce_llama_ray.sh